In [6]:
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [8]:
import pandas as pd

# 엑셀 읽기
path = "market_stats.xlsx"
df = pd.read_excel(path)

df.columns = ["month", "region", "passenger", "van", "truck", "special"]

# 1) month NaN → 위 값으로 채우기
df["month"] = df["month"].ffill()

# 🚨 2) 최상단에 있는 "계" 문자열 행 제거
df = df[df["region"].notna()]          # region이 NaN인 행 제거
df = df[df["passenger"] != "계"]        # '계' 라벨 제거
df = df[df["region"] != "시군구"]       # 혹시 header가 중복되면 제거

# 3) 숫자 데이터 정제
for col in ["passenger", "van", "truck", "special"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(int)

# 4) 시군구 == '합계' 만 추출
df_sum = df[df["region"] == "합계"].copy()

# 5) 정규화
normalized = []
vehicle_map = {
    "passenger": "승용",
    "van": "승합",
    "truck": "화물",
    "special": "특수"
}

for _, row in df_sum.iterrows():
    year_month = row["month"]
    
    for col, vtype in vehicle_map.items():
        normalized.append({
            "year_month": year_month,
            "vehicle_type": vtype,
            "registration_cnt": int(row[col])
        })

result_df = pd.DataFrame(normalized)
result_df


,year_month,vehicle_type,registration_cnt
0,2023-01,승용,2765782
1,2023-01,승합,95828
2,2023-01,화물,321896
3,2023-01,특수,10845
4,2023-01,승용,1264858
...,...,...,...
2307,2025-10,특수,5653
2308,2025-10,승용,777407
2309,2025-10,승합,27197
2310,2025-10,화물,194438


In [ ]:
import pandas as pd

# 엑셀 읽기
path = "market_stats.xlsx"   # 파일명에 맞게 변경
df = pd.read_excel(path)

# 컬럼 이름 정리
df.columns = ["month", "region", "passenger", "van", "truck", "special"]

# 1) 월(month) NaN → 위의 값으로 채움
df["month"] = df["month"].ffill()

# 2) 데이터가 아닌 row 제거 (불필요한 '계', NaN 등)
df = df[df["region"].notna()]          # region == NaN 제거
df = df[df["passenger"] != "계"]        # passenger 열의 '계' 제거
df = df[df["region"] != "시군구"]       # header 중복 제거 가능성 대비

# 3) 숫자 데이터 정제 (쉼표 제거 → int)
for col in ["passenger", "van", "truck", "special"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(int)

# 4) 시군구 == '합계' 인 행만 추출
df_sum = df[df["region"] == "합계"].copy()

# 5) 정규화
normalized = []
vehicle_map = {
    "passenger": "승용",
    "van": "승합",
    "truck": "화물",
    "special": "특수"
}

for _, row in df_sum.iterrows():
    year_month = row["month"]

    for col, vtype in vehicle_map.items():
        normalized.append({
            "year_month": year_month,
            "vehicle_type": vtype,
            "registration_cnt": int(row[col])
        })

# 6) 결과 DataFrame으로 변환
result_df = pd.DataFrame(normalized)

# 7) CSV로 저장
output_path = "vehicle_registration_normalized.csv"
result_df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"CSV 저장 완료 → {output_path}")


CSV 저장 완료 → vehicle_registration_normalized.csv


In [11]:
import pandas as pd

df = pd.read_csv("vehicle_registration_normalized.csv")

# year_month + vehicle_type 기준으로 registration_cnt 합계
df_grouped = (
    df.groupby(["year_month", "vehicle_type"], as_index=False)["registration_cnt"]
      .sum()
)

df_grouped

df_grouped.to_csv("vehicle_registration_monthly_sum.csv", index=False, encoding="utf-8-sig")
